# **Ejercicio 2**

## **EDA + ETL** 
Dado los siguientes conjuntos de datos `NeurIPS 2024- Predict New Medicines with BELKA` y `Open Problems– Single-Cell Perturbations`, realizar un análisis exploratorio de datos el cual incluya lo siguiente:

- Análisis del problema de aplicación. Realice un resumen describiendo la importancia del problema, en que consiste y la fuente de los datos. Realice descripción del tipos de variables, calcular *número de observaciones, media, desviación estándar, mínimo, máximo, cuartiles*, realizar conteo de *datos faltantes y su porcentaje, histograma o diagrama de barras para la variable respuesta e independientes según corresponda*. Análisis de *simetría, datos atípicos y dispersión, etc,...* por medio de `boxplot()`. *Análisis bivariado*. `Trazado de scatterplot() y regplot()`. En cada figura agregar un análisis y descripción. Complemente el EDA con visualizaciones que muestren patrones importantes presentes en los datos. 

- Según corresponda, realizar imputación de datos faltantes usando imputación múltiple iteraticva (ver `IterativeImputer()`).Realizar reducción de dimensionalidad por medio de eliminiación de columnas altamente correlacionadas usando *Variance Inflation Factor (VIF)*. Para esto se recomienda usar la siguiente librería `variance_inflation_factor()`. Un *VIF ≥ 10* indica alta multicolinealidad entre una variable independiente y mas de dos variables explicativas. 

Recomendación:

Eliminar una columna a la vez. Aquella con el máximo VIF ≥ 10. Luego, para el nuevo pandas, calcular nuevamente VIF e identificar nuevas columnas con VIF ≥ 10 máximo, y así sucesivamente hasta obtener solo valores de VIF < 10. Según corresponda, variables categóricas deben previamente codificarse usando por ejemplo `OneHotEncoder()` ver por ejemplo Aplicación: Titanic Dataset. Pueden mantener las variables categóricas antes de la codificación previa al entrenamiento del modelo y reducir multicolinealidad usando la prueba `chi2_contingency()`. Si el número de variables explicativas es pequeño y además, cada una de las variables son de gran importancia para explicar las predicciones, una alternativa correcta es mantenerlas todas. Como Científico de Datos, tome la decisión más adecuada y justifíquela.